# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# read file
df = pd.read_csv(output_data_file)

# print data frame
df

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,Tasiilaq,GL,2020-03-17 16:20:13,65.61,-37.64,20,78,14.00,3.36
1,Vao,NC,2020-03-17 16:20:13,-22.67,167.48,26,76,76.55,18.81
2,Kirkuk,IQ,2020-03-17 16:20:13,35.47,44.39,0,62,59.63,14.52
3,Ushuaia,AR,2020-03-17 16:18:17,-54.80,-68.30,75,50,55.40,3.36
4,Fairbanks,US,2020-03-17 16:20:03,64.84,-147.72,1,73,34.00,6.93
...,...,...,...,...,...,...,...,...,...
547,Batagay,RU,2020-03-17 16:20:45,67.63,134.63,100,99,-6.07,2.42
548,Belém de São Francisco,BR,2020-03-17 16:20:45,-8.75,-38.97,100,55,85.91,4.05
549,Jalu,LY,2020-03-17 16:20:45,29.03,21.55,0,28,62.87,16.24
550,Karla,EE,2020-03-17 16:20:45,59.37,24.93,90,80,43.00,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = df[['Latitude', 'Longitude']]
weights = df['Humidity']
fig = gmaps.figure(layout = figure_layout)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

hotel_df = df.loc[(df["Max Temperature"] >= 70) & (df["Max Temperature"] <= 80) &
                    (df["Wind Speed"] <= 10) & (df["Cloudiness"] == 0)]

hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
205,Castro,BR,2020-03-17 16:20:24,-24.79,-50.01,0,80,72.05,5.39
213,Makkah al Mukarramah,SA,2020-03-17 16:20:25,21.43,39.83,0,48,73.85,0.36
223,Basna,IN,2020-03-17 16:20:25,21.28,82.82,0,34,76.37,2.33
276,Raja,SS,2020-03-17 16:20:28,8.46,25.68,0,22,77.81,3.85
300,Māngrol,IN,2020-03-17 16:20:30,21.12,70.12,0,64,78.71,9.95
336,Inhambane,MZ,2020-03-17 16:20:32,-23.86,35.38,0,73,73.40,6.93
363,Dhandhuka,IN,2020-03-17 16:20:34,22.37,71.98,0,20,75.65,6.80
364,Bima,ID,2020-03-17 16:20:34,-8.47,118.72,0,94,71.51,2.73
445,Abu Dhabi,AE,2020-03-17 16:20:39,24.47,54.37,0,64,77.00,4.70
473,Marsh Harbour,BS,2020-03-17 16:20:41,26.54,-77.06,0,67,72.77,9.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df["Hotel Name"] = ""

# create a params dict that will be updated with new city each iteration
params = {
    "key": g_key,
    "radius": 5000,
    "keyword": "hotel",
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    lat = row['Latitude']
    lng = row['Longitude']

    # update address key value
    params['location'] = f"{lat},{lng}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
        
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
        
    try:
        # fill in data frame
        hotel_df.loc[index, 'Hotel Name'] = cities_lat_lng['results'][0]['name']
        
    except:
        print(f"Skip row {index}")

# Drop missing rows and print data frame
hotel_df = hotel_df.drop([276,525,546])
hotel_df

C:\Users\swati\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\swati\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Skip row 276
Skip row 525
Skip row 546


,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel Name
205,Castro,BR,2020-03-17 16:20:24,-24.79,-50.01,0,80,72.05,5.39,Hotel Buganville Express
213,Makkah al Mukarramah,SA,2020-03-17 16:20:25,21.43,39.83,0,48,73.85,0.36,Pullman Zamzam Makkah
223,Basna,IN,2020-03-17 16:20:25,21.28,82.82,0,34,76.37,2.33,Hotel Grand Alfa [BASNA]
300,Māngrol,IN,2020-03-17 16:20:30,21.12,70.12,0,64,78.71,9.95,Kaveri Hotel
336,Inhambane,MZ,2020-03-17 16:20:32,-23.86,35.38,0,73,73.40,6.93,Hotel Casa Do Capitão
363,Dhandhuka,IN,2020-03-17 16:20:34,22.37,71.98,0,20,75.65,6.80,Sakti T Sentar
364,Bima,ID,2020-03-17 16:20:34,-8.47,118.72,0,94,71.51,2.73,Tambora Homestay Bima
445,Abu Dhabi,AE,2020-03-17 16:20:39,24.47,54.37,0,64,77.00,4.70,Jumeirah At Etihad Towers
473,Marsh Harbour,BS,2020-03-17 16:20:41,26.54,-77.06,0,67,72.77,9.01,Abaco Beach Resort
511,Port Hedland,AU,2020-03-17 16:20:43,-20.32,118.57,0,88,78.80,4.70,The Esplanade Hotel


In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = df[['Latitude', 'Longitude']]
weights = df['Humidity']
fig = gmaps.figure(layout = figure_layout)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

# Assign the marker layer to a variable
hotel_locations = hotel_df[['Latitude', 'Longitude']]
markers = gmaps.marker_layer(hotel_locations)

# Add the layer to the map
fig.add_layer(markers)

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
symbol_layer = gmaps.symbol_layer(hotel_locations, hover_text=hotel_info)
fig.add_layer(symbol_layer)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…